In [1]:

from pathlib import Path

def _start_or_end_with(text, pattern):
  pattern = tuple(pattern)
  return text.startswith(pattern) or text.endswith(pattern)

def load_csv(path, sample=10, only_title=False, include=(), exclude=()):
  from itertools import compress
  import csv

  with open(path) as f:
    titles = f.readline().strip().split(',')

  if include:
    column_compress = [_start_or_end_with(title, include) for title in titles]
  else:
    column_compress = [True] * len(titles)
  if exclude:
    column_compress = [not _start_or_end_with(title, exclude) and tb for title, tb in zip(titles, column_compress)]

  if only_title:
    return list(compress(titles, column_compress))

  with open(path) as f:
    lines = csv.reader(f)
    next(lines)
    result = []
    for i, line in enumerate(lines, 1):
      if sample and i > sample:
        break
      # result.append([_load_csv_value_convert(x) for x in compress(line, column_compress)])
      result.append([x for x in compress(line, column_compress)])

  return result



    目录结构大致是
    .
    |____cancer mortality white male 65 - sheet1.csv
    |____ewg_process.ipynb
    |____epa_water_system
    | |____epa_water_system_summary_AK_[3549].csv
    | |____epa_water_system_summary_AL_[1810].csv
    | |____epa_water_system_summary_AR_[4194].csv
    | |____epa_water_system_summary_AS_[273].csv
    | |____epa_water_system_summary_AZ_[3967].csv
    | |____epa_water_system_summary_CA_[18131].csv
    | |____epa_water_system_summary_CO_[4588].csv
    | |____epa_water_system_summary_CT_[11043].csv
    | |____epa_water_system_summary_DE_[1300].csv
    | |____......
    | |____......
    | |____......
    | |____epa_water_system_summary_TN_[4939].csv
    | |____epa_water_system_summary_TX_[15282].csv
    | |____epa_water_system_summary_UT_[2114].csv
    | |____epa_water_system_summary_VA_[8721].csv
    | |____epa_water_system_summary_VI_[986].csv
    | |____epa_water_system_summary_VT_[4286].csv
    | |____epa_water_system_summary_WA_[9465].csv
    | |____epa_water_system_summary_WI_[23678].csv
    | |____epa_water_system_summary_WV_[4337].csv
    | |____epa_water_system_summary_WY_[1859].csv
    | 
    |____ewg_all_done[22466]_nodata[396908].csv



### 找出 `cancer mortality white male 65.xls` 中所有需要处理的 州-县

In [29]:
sheet = './cancer mortality white male 65 - sheet1.csv'
headers = load_csv(sheet, only_title=True)
print('headers: ', headers)


county_list = load_csv(sheet, sample=None, include=('County', ))

# 略过形如 "Strafford County, New Hampshire (6, 7)" 的括号
county_list = [elem[0].split(' (')[0] for elem in county_list if elem[0] != 'United States (6)']

print('\n\n')
print('counties: ', county_list[:20], '...')

headers:  ['County', ' FIPS', 'Met Healthy People Objective of 161.4?', '"Age-Adjusted Death Rate(? - deaths per 100', '000"', 'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval', 'Average Annual Count', 'Recent Trend', 'Recent 5-Year Trend (? in Death Rates', 'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval', '', 'life expenctancy', '']



counties:  ['Douglas County, Colorado', 'Fairfax County, Virginia', 'Marin County, California', 'Santa Clara County, California', 'Loudoun County, Virginia', 'Collier County, Florida', 'Howard County, Maryland', 'San Mateo County, California', 'Bergen County, New Jersey', 'Olmsted County, Minnesota', 'Stearns County, Minnesota', 'Westchester County, New York', 'Hunterdon County, New Jersey', 'Orange County, California', 'Rockland County, New York', 'Nassau County, New York', 'Morris County, New Jersey', 'Somerset County, New Jersey', 'Washington County, Minnesota', 'Ozaukee County, Wisconsin'] ...


### 找出 `epa_water_system/` 中对应 州-县 的所有水厂 ID

In [52]:
state_dict = '''阿拉巴马州	Alabama	AL	蒙哥马利	Montgomery
阿拉斯加州	Alaska	AK	朱诺	Juneau
阿利桑那州	Arizona	AZ	菲尼克斯	Phoenix
阿肯色州	Arkansas	AR	小石城	Little rock
加利福尼亚州	California	CA	萨克拉门托	Sacramento
科罗拉多州	Colorado	CO	丹佛	Denver
康涅狄格州	Connecticut	CT	哈特福德	Hartford
特拉华州	Delaware	DE	多佛	Dover
佛罗里达州	Florida	FL	塔拉哈西	Tallahassee
乔治亚州	Georgia	GA	亚特兰大	Atlanta
夏威夷州	Hawaii	HI	檀香山www.fltacn.com火努鲁鲁	Honolulu
爱达荷州	Idaho	ID	博伊西	Boise
伊利诺斯州	Illinois	IL	斯普林菲尔德	Springfield
印第安纳州	Indiana	IN	印第安纳波利斯	Indianapolis
爱荷华州	Iowa	IA	得梅因	Des Moines
堪萨斯州	Kansas	KS	托皮卡	Topeka
肯塔基州	Kentucky	KY	法兰克福	Frankfort
路易斯安那州	Louisiana	LA	巴吞鲁日	Baton Rouge
缅因州	Maine	ME	奥古斯塔	Augusta
马里兰州	Maryland	MD	安纳波利斯	Annapolis
马萨诸塞州	Massachusetts	MA	波士顿	Boston
密歇根州	Michigan	MI	兰辛	Lansing
明尼苏达州	Minnesota	MN	圣保罗	St. Paul
密西西比州	Mississippi	MS	杰克逊	Jackson
密苏里州	Missouri	MO	杰斐逊城	Jefferson City
蒙大拿州	Montana	MT	海伦娜	Helena
内布拉斯加州	Nebraska	NE	林肯	Lincoln
内华达州	Nevada	NV	卡森城	Carson City
新罕布什尔州	New Hampshire	NH	康科德	Concord
新泽西州	New Jersey	NJ	特伦顿	Trenton
新墨西哥州	New Mexico	NM	圣菲	Santa Fe
纽约州	New York	NY	奥尔巴尼	Albany
北卡罗来纳州	North Carolina	NC	纳罗利	Raleigh
北达科他州	North Dakota	ND	俾斯麦	Bismarck
俄亥俄州	Ohio	OH	哥伦布	Columbus
俄克拉荷马州	Oklahoma	OK	俄克拉何马城	Oklahoma City
俄勒冈州	Oregon	OR	塞勒姆	Salem
宾夕法尼亚州	Pennsylvania	PA	哈里斯堡	Harrisburg
罗得岛州	Rhode Island	RI	普罗维登斯	Providence
南卡罗来纳州	South Carolina	SC	哥伦比亚	Columbia
南达科他州	South Dakota	SD	皮尔	Pierre
田纳西州	Tennessee	TN	纳什维尔	Nashville
得克萨斯州	Texas	TX	奥斯汀	Austin
犹他州	Utah	UT	盐湖城	Salt Lake City
佛蒙特州	Vermont	VT	蒙彼利埃	Montpelier
弗吉尼亚州	Virginia	VA	里士满	Richmond
华盛顿州	Washington	WA	奥林匹亚	Olympia
西弗吉尼亚州	West Virginia	WV	查尔斯顿	Charleston
威斯康辛州	Wisconsin	WI	麦迪逊	Madison
怀俄明州	Wyoming	WY	夏延	Cheyenne'''

state_dict = {line.split('\t')[1]: line.split('\t')[2] for line in state_dict.splitlines()}


alpha_num_code_dict = 'AK 02,AL 01,AR 05,AZ 04,CA 06,CO 08,CT 09,DE 10,FL 12,GA 13,HI 15,IA 19,ID 16,IL 17,IN 18,KS 20,KY 21,LA 22,MA 25,MD 24,ME 23,MI 26,MN 27,MO 29,MS 28,MT 30,NC 37,ND 38,NE 31,NH 33,NJ 34,NM 35,NV 32,NY 36,OH 39,OK 40,OR 41,PA 42,RI 44,SC 45,SD 46,TN 47,TX 48,UT 49,VA 51,VT 50,WA 53,WI 55,WV 54,WY 56'
alpha_num_code_dict = {alnum[:2]: alnum[3:] for alnum in alpha_num_code_dict.split(',')}

def find_epa_water_system_csv_filename(state_county_name):
  # 以 州-县 名称找到对应的 epa_water_system*.csv 文件名
  # 如 epa_water_system_summary_CO_[4588].csv
  
  state_name = state_county_name.split(',')[1].strip()
  state_alias = state_dict[state_name]
  # filename like epa_water_system_summary_OR_[5802].csv
  ret = list(Path('./epa_water_system/').glob('epa_water_system_summary_{}*.csv'.format(state_alias)))
  if len(ret) != 1:
    raise ValueError
  else:
    return str(ret[0])
  
  
def find_pwsids_from_epa_water_system_csv(state_county_name, epa_water_system_csv):
  # 以 州-县 名称 + epa_water_system*.csv 文件名, 
  # 找到给该县供水的全部 pwsids 和对应人口
  # 返回 (pwsid1, pop1), (pwsid2, pop2), ... 列表
  county_name = state_county_name.split(',')[0].strip().replace(' County', '')
  # print(county_name)
  epa_data = load_csv(epa_water_system_csv, sample=None, include='PWS ID,Counties Served,Population Served Count'.split(','))
  return [(pwsid, int(pop_count.replace(',', ''))) for pwsid, name, pop_count in epa_data if name == county_name]

In [31]:
# 测试用
state_county_name = 'Douglas County, Colorado'
state_county_name = 'Adams County, Colorado'
# state_county_name = 'Stearns County, Minnesota'

epa_water_system_csv = find_epa_water_system_csv_filename(state_county_name)
print('find epa_water_system_csv', epa_water_system_csv)

county_pwsid_and_pop_count_list = find_pwsids_from_epa_water_system_csv(state_county_name, epa_water_system_csv)
print(county_pwsid_and_pop_count_list[:20], '...', len(county_pwsid_and_pop_count_list))

find epa_water_system_csv epa_water_system/epa_water_system_summary_CO_[4588].csv
[('CO0101001', 0), ('CO0053876', 200), ('CO0101005', 0), ('CO0101010', 0), ('CO0201006', 257), ('CO0028969', 85), ('CO0101015', 460), ('CO0101020', 3100), ('CO0116140', 3396), ('CO0117358', 101), ('CO0101025', 36010), ('CO0101030', 77), ('CO0101166', 150), ('CO0101035', 0), ('CO0101185', 1000), ('CO0101188', 160), ('CO0101040', 17000), ('CO0101187', 100), ('CO0101205', 1200), ('CO0101234', 1193)] ... 165


### 用对应 州-县 的所有水厂 ID 找到 Nitrate 数据, 大部分应该是没有数据

In [32]:
nitrate_data = {}


# 计算一次就行, 很慢
for pwsid, state, national, this_utility in load_csv('./ewg_all_done[22466]_nodata[396908].csv', sample=None):
  if this_utility != 'nodata':
    nitrate_data[pwsid] = float(this_utility.replace(' ppm', ''))

len(nitrate_data.keys())

22466

### 对有数据的部分做加权平均, 得到该 州-县 的 Nitrate 水平

In [33]:

# 测试用
county_nitrate_list = [(pop_count, nitrate_data[pwsid]) for pwsid, pop_count in county_pwsid_and_pop_count_list if pwsid in nitrate_data]


print(county_nitrate_list)

print()
print(state_county_name, '- mean_nitrate_value: ', sum(pop*val for pop, val in county_nitrate_list) / sum(pop for pop, val in county_nitrate_list))

[(36010, 4.22), (77, 6.78), (1000, 0.011), (1200, 0.011), (1500, 4.22), (11678, 0.011), (961, 1.9), (986, 0.011), (2005, 0.011), (651, 0.011), (130, 0.169), (1965, 0.48), (70, 0.2), (58737, 4.88), (2050, 1.5), (136977, 0.48), (5078, 1.5), (178250, 0.15)]

Adams County, Colorado - mean_nitrate_value:  1.255624127923519


In [34]:
# 打印 csv header 部分
print('state_county,pwsid_count,pwsid_with_nitrate_count,ave_nitrate_index')


for state_county_name in county_list:
  if state_county_name == 'District of Columbia':  # 哪里的自来水厂给 D.C. 供水? 暂略过
    print('"{state_county_name}",??,??,??'.format(**locals()))
    continue
  

  epa_water_system_csv = find_epa_water_system_csv_filename(state_county_name)
  
  # print('find epa_water_system_csv', epa_water_system_csv)

  # (pwsid1, pop1), (pwsid2, pop2), ... 列表, 含该县全部已知的水厂
  county_pwsid_and_pop_count_list = find_pwsids_from_epa_water_system_csv(state_county_name, epa_water_system_csv)
  
  # print(county_pwsid_and_pop_count_list[:20], '...', len(county_pwsid_and_pop_count_list))
  
  pwsid_count = len(county_pwsid_and_pop_count_list)
  
  # (pop1, nitrate_index1), (pop2, nitrate_index2), ... 列表, 仅含有 nitrate 数据的那部分
  county_nitrate_list = [(pop_count, nitrate_data[pwsid]) for pwsid, pop_count in county_pwsid_and_pop_count_list if pwsid in nitrate_data]
  pwsid_with_nitrate_count = len(county_nitrate_list)
  
  # print(county_nitrate_list)
  
  try:
    # 按人口加权平均 nitrate 指数
    ave_nitrate_index = sum(pop*val for pop, val in county_nitrate_list) / sum(pop for pop, val in county_nitrate_list)
  except ZeroDivisionError:  # 带 nitrate 数据的水厂数为 0
    ave_nitrate_index = -1
  
  print('"{state_county_name}",{pwsid_count},{pwsid_with_nitrate_count},{ave_nitrate_index:.4f}'.format(**locals()))


state_county,pwsid_count,pwsid_with_nitrate_count,ave_nitrate_index
"Douglas County, Colorado",117,14,0.1007
"Fairfax County, Virginia",198,4,0.9240
"Marin County, California",103,8,0.1217
"Santa Clara County, California",318,53,3.2266
"Loudoun County, Virginia",180,1,2.7000
"Collier County, Florida",151,12,0.0694
"Howard County, Maryland",179,1,4.7200
"San Mateo County, California",113,11,2.2623
"Bergen County, New Jersey",734,13,2.8138
"Olmsted County, Minnesota",208,3,0.0605
"Stearns County, Minnesota",742,19,0.6293
"Westchester County, New York",348,77,0.2502
"Hunterdon County, New Jersey",1016,20,1.9390
"Orange County, California",167,33,1.7827
"Rockland County, New York",103,6,1.7763
"Nassau County, New York",48,28,2.8343
"Morris County, New Jersey",426,29,1.3235
"Somerset County, New Jersey",146,3,0.7447
"Washington County, Minnesota",545,21,1.1036
"Ozaukee County, Wisconsin",526,14,0.5567
"DuPage County, Illinois",1314,8,0.1056
"Dakota County, Minnesota",479,10,0.9424
"Fairfiel

In [59]:
from collections import OrderedDict as od

folder = Path('C:/Users/Probe/Downloads/Cancer risk factors')

nitrate_index_csv = Path('./cancer mortality white male 65 - sheet2 - nitrate_index.csv')

load_csv(str(nitrate_index_csv), only_title=True)

nitrate_index_data = load_csv(str(nitrate_index_csv), sample=None)

print(nitrate_index_data)

# smoking = folder / 'total smoking prevalance M 2009.csv'
# print('smoking', load_csv(str(smoking), only_title=True))
# print('smoking', load_csv(str(smoking), only_title=False, sample=20))
# smoking_data = od((c + ', ' + s, i) for s, c, _, _, i in load_csv(str(smoking), sample=None))
# smoking_data


# alcohol = folder / 'alchol use total M 2009.csv'
# print('alcohol', load_csv(str(alcohol), only_title=True))
# print('alcohol', load_csv(str(alcohol), only_title=False, sample=20))
# alcohol_data = od((c + ', ' + s, i) for s, c, i in load_csv(str(alcohol), sample=20))
# alcohol_data


# diabete = folder / 'diabetes prevalence total M 2009.csv'
# print('diabete', load_csv(str(diabete), only_title=True))
# print('diabete', load_csv(str(diabete), only_title=False, sample=20))
# invert_state_dict = {v: k for k, v in state_dict.items()}
# num_2_state_dict = {int(v): invert_state_dict[k] for k, v in alpha_num_code_dict.items()}
# diabete_data = od((c + ', ' + num_2_state_dict[int(num[:-3])], i) for c, num, i in load_csv(str(diabete), sample=10) if num and num.isdigit() and len(num)>3)
# diabete_data

# physical_activity = folder / 'total_OBESITY_PHYSICAL_ACTIVITY_M 2009.csv'
# print('physical_activity', load_csv(str(physical_activity), only_title=True))
# print('physical_activity', load_csv(str(physical_activity), only_title=False, sample=20))
# physical_activity_data = od((c.split(',')[0] + ' County, ' + s, i) for _, _, s, c, _, _, i in load_csv(str(physical_activity), sample=20))
# physical_activity_data


mortality = folder / 'cancer mortality white male 65+.csv'
print('mortality', load_csv(str(mortality), only_title=True))
print('mortality', load_csv(str(mortality), only_title=False, sample=20))
mortality_data = od((sc, d) for sc, f, d, _, l, pop in load_csv(str(mortality), sample=20))
mortality_data





[['Douglas County, Colorado', '117', '14', '0.1007'], ['Fairfax County, Virginia', '198', '4', '0.9240'], ['Marin County, California', '103', '8', '0.1217'], ['Santa Clara County, California', '318', '53', '3.2266'], ['Loudoun County, Virginia', '180', '1', '2.7000'], ['Collier County, Florida', '151', '12', '0.0694'], ['Howard County, Maryland', '179', '1', '4.7200'], ['San Mateo County, California', '113', '11', '2.2623'], ['Bergen County, New Jersey', '734', '13', '2.8138'], ['Olmsted County, Minnesota', '208', '3', '0.0605'], ['Stearns County, Minnesota', '742', '19', '0.6293'], ['Westchester County, New York', '348', '77', '0.2502'], ['Hunterdon County, New Jersey', '1016', '20', '1.9390'], ['Orange County, California', '167', '33', '1.7827'], ['Rockland County, New York', '103', '6', '1.7763'], ['Nassau County, New York', '48', '28', '2.8343'], ['Morris County, New Jersey', '426', '29', '1.3235'], ['Somerset County, New Jersey', '146', '3', '0.7447'], ['Washington County, Minneso

OrderedDict([('United States ', '1216.9'),
             ('Douglas County, Colorado', '984.6'),
             ('Fairfax County, Virginia', '937.7'),
             ('Marin County, California', '966.4'),
             ('Santa Clara County, California', '1107.9'),
             ('Loudoun County, Virginia', '1036.8'),
             ('Collier County, Florida', '787.9'),
             ('Howard County, Maryland', '1008.8'),
             ('San Mateo County, California', '1119.1'),
             ('Bergen County, New Jersey', '1111.8'),
             ('Olmsted County, Minnesota', '1055.7'),
             ('Stearns County, Minnesota', '1167.8'),
             ('Westchester County, New York', '1065'),
             ('Hunterdon County, New Jersey', '1034.9'),
             ('Orange County, California', '1139.7'),
             ('Rockland County, New York', '993.7'),
             ('Nassau County, New York', '1057.6'),
             ('Morris County, New Jersey', '1148.7'),
             ('Somerset County, New Jersey